# データをDataFrame形式に成形

DL元：https://www.rondhuit.com/download.html  
ldcc-20140209.tar.gzというやつをダウンロード。落としてきたディレクトリで、

と打てば解凍できる。同じディレクトリに"text"というフォルダが生成されて、項目別にニュースのデータ(.txt形式)が入っている。

In [ ]:
import glob
from tqdm import tqdm
import pandas as pd
import os

#ファイルのリストを取得
filepath = './nlp/livedoor-news/' #データが入ってるディレクトリ
news_list = glob.glob(filepath+'**/*.txt')

#dfの外枠を作っておく
origin_df = pd.DataFrame(columns=['URL','news_date','news_title','main_doc','category'],index=range(len(news_list)))

#1つずつ読み込んでdfに追加していく
for i in tqdm(range(len(news_list))):
    with open(news_list[i],'r') as f:
        text = f.read()
        df_list = text.split('\n')[:3]
        main_article = ''.join(text.split('\n')[3:])
        df_list.append(main_article)
        category = os.path.dirname(news_list[i]).replace(filepath,'')
        df_list.append(category)
        origin_df.iloc[i,:] = df_list

ここでできたorigin_dfがDataFrame形式のデータになっている。  
この時点で保存するなら以下のような感じ

In [ ]:
#dfをpickleで保存
import pickle
with open('./livedoor-news.pickle','wb') as f:
    pickle.dump(origin_df,f)

In [ ]:
#dfをtsv形式で保存
origin_df.to_csv('livedoor-news.tsv',sep='\t',index=False)

In [ ]:
#pickleを読み出す
import pickle
with open('./livedoor-news.pickle','rb') as f:
        df = pickle.load(f)

## カテゴリをラベルに変換する

In [ ]:
#フォルダ名のリスト（=カテゴリ名のリスト）を作成
import glob
category_list = glob.glob('./nlp/livedoor-news/*')
filepath = './nlp/livedoor-news/'

for i in range(len(category_list)):
    new_path = category_list[i].replace(filepath,'')
    category_list[i] = new_path
    
category_list = [path for path in category_list if '.txt' not in path]
print(category_list)

In [ ]:
#categoryを数字ラベルに変換
df['label'] = 0 #新しくDataFrameに"label"の列を追加
for i in range(len(df)):
    index = category_list.index(df.iloc[i,4])
    df.iloc[i,5] = index #"label"列にラベルを上書きしていく

print(df.head())

## train/val/testとして保存

In [ ]:
#ランダムシャッフル
df = df.sample(frac=1).reset_index(drop=True)

今回必要なのは記事の中身(main_doc)とカテゴリのラベル(label) なので、そこだけ取り出して保存しておく

In [ ]:
#訓練、検証、テストに分けてそれぞれtsvファイルで保存
train_livedoor = df.iloc[0:5000,[3,5]]
val_livedoor = df.iloc[5000:6000,[3,5]]
test_livedoor = df.iloc[6000:7000,[3,5]]
train_livedoor.to_csv('train_livedoor.tsv',sep='\t',index=False)
val_livedoor.to_csv('val_livedoor.tsv',sep='\t',index=False)
test_livedoor.to_csv('test_livedoor.tsv',sep='\t',index=False)